In [2]:
import numpy as np
from os import listdir

def img2vector(directory, filename):
    vector1 = np.zeros((1, 1024))
    vector2 = np.zeros((1, 256))
    data = np.zeros((32, 32))
    f = open(directory + '/' + filename)
    for i in range(32):
        line = f.readline()
        for j in range(32):
            data[i, j] = int(line[j])
            vector1[0, 32*i+j] = int(line[j])
    for i in range(0, 32, 2):
        for j in range(0, 32, 2):
            vector2[0, int(i*8+j/2)] = data[i][j]+data[i][j+1]+data[i+1][j]+data[i+1][j+1]
    return vector1, vector2

def getData(filename):
    fileList = listdir(filename)
    m = len(fileList)
    dataset = np.zeros((m, 1024))
    dataset_DR = np.zeros((m, 256))
    labels = []
    for i in range(m):
        labels.append(int(fileList[i].split('_')[0]))
        dataset[i], dataset_DR[i] = img2vector(filename, fileList[i])
    return dataset, dataset_DR, labels

X_train, X_train_DR, y_train = getData('trainingDigits')
X_test, X_test_DR, y_test = getData('testDigits')

In [11]:
import numpy as np
np.set_printoptions(threshold=np.inf)

def train(trainMatrix, trainCategory):
    vectorNum = len(trainMatrix)
    featuresNum = len(trainMatrix[0])
    labelNum = len(set(trainCategory))
    pA = np.zeros(labelNum)
    pB = np.ones((labelNum, featuresNum))                                      
    for i in range(vectorNum):
        pB[trainCategory[i]] += trainMatrix[i]
    for i in range(labelNum):
        pA[i] = trainCategory.count(i)
        pB[i] /= pA[i]+2
    return pB, pA

def classify(vecClassify, pB, pA):
    pA_B = np.zeros(len(pA))
    for index in range(len(pA)):
        p = 0.0
        for i, j in zip(vecClassify, pB[index]):
            p += np.log(j) if i == 1 else np.log(1.0-j)
        p += np.log(pA[index])
        pA_B[index] = p
    return np.argmax(pA_B)

def score(X_train, y_train, X_test, y_test):
    pB, pA = train(X_train, y_train)
    error = 0
    for X, y in zip(X_test, y_test):
        if classify(X, pB, pA) != y:
            error += 1
    print('error rate: %.2f%%' % (error/float(len(y_test)) * 100))
if __name__ == '__main__':
	score(X_train, y_train, X_test, y_test)

error rate: 6.87%
